# Function Calling in OpenAI

In [1]:
import os
import openai
from dotenv import load_dotenv

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
prompt = "Hello, How are you?"

response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response["choices"][0]["message"]["content"])

Hello! I'm here and ready to assist you. How can I help you today?


https://rapidapi.com/hub

In [5]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

import requests
def get_current_weather(location):
    """Get the current weather in a given location"""

    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"

    querystring = {"text":location}

    headers = {
      'x-rapidapi-key': "156877d1e1msh6806b57dfec44b9p1bd2c8jsn796f2ff15790",
      'x-rapidapi-host': "ai-weather-by-meteosource.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())
  
    return response.json()


In [6]:
response = get_current_weather("Delhi")

[{'name': 'Delhi', 'place_id': 'delhi', 'adm_area1': 'National Capital Territory of Delhi', 'adm_area2': None, 'country': 'India', 'lat': '28.65195N', 'lon': '77.23149E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Dili', 'place_id': 'dili', 'adm_area1': 'Díli', 'adm_area2': 'Vera Cruz', 'country': 'Timor-Leste', 'lat': '8.55861S', 'lon': '125.57361E', 'timezone': 'Asia/Dili', 'type': 'settlement'}, {'name': 'Delhi', 'place_id': 'delhi-5342522', 'adm_area1': 'California', 'adm_area2': 'Merced', 'country': 'United States of America', 'lat': '37.43216N', 'lon': '120.77854W', 'timezone': 'America/Los_Angeles', 'type': 'settlement'}, {'name': 'Delhi', 'place_id': 'delhi-5114824', 'adm_area1': 'New York', 'adm_area2': 'Delaware', 'country': 'United States of America', 'lat': '42.27814N', 'lon': '74.91599W', 'timezone': 'America/New_York', 'type': 'settlement'}, {'name': 'Delhi', 'place_id': 'delhi-4321929', 'adm_area1': 'Louisiana', 'adm_area2': 'Richland', 'country': 'Unit

In [7]:
response

[{'name': 'Delhi',
  'place_id': 'delhi',
  'adm_area1': 'National Capital Territory of Delhi',
  'adm_area2': None,
  'country': 'India',
  'lat': '28.65195N',
  'lon': '77.23149E',
  'timezone': 'Asia/Kolkata',
  'type': 'settlement'},
 {'name': 'Dili',
  'place_id': 'dili',
  'adm_area1': 'Díli',
  'adm_area2': 'Vera Cruz',
  'country': 'Timor-Leste',
  'lat': '8.55861S',
  'lon': '125.57361E',
  'timezone': 'Asia/Dili',
  'type': 'settlement'},
 {'name': 'Delhi',
  'place_id': 'delhi-5342522',
  'adm_area1': 'California',
  'adm_area2': 'Merced',
  'country': 'United States of America',
  'lat': '37.43216N',
  'lon': '120.77854W',
  'timezone': 'America/Los_Angeles',
  'type': 'settlement'},
 {'name': 'Delhi',
  'place_id': 'delhi-5114824',
  'adm_area1': 'New York',
  'adm_area2': 'Delaware',
  'country': 'United States of America',
  'lat': '42.27814N',
  'lon': '74.91599W',
  'timezone': 'America/New_York',
  'type': 'settlement'},
 {'name': 'Delhi',
  'place_id': 'delhi-4321929

In [8]:
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    
                },
                "required": ["location"],
            },
        }
    ]

In [9]:

user_message="Hi There"
messages=[]
messages.append({"role": "user", "content":user_message})

completion=openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages= messages
    
)

In [10]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "Hello! How can I assist you today?",
  "refusal": null
}


In [11]:
messages

[{'role': 'user', 'content': 'Hi There'}]

In [36]:
user_message="What is the temperature of Bangalore"

messages.append({"role": "user", "content": user_message})

completion=openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
    
)

In [37]:
# messages

In [38]:
# completion

In [39]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\":\"Bangalore\"}"
  },
  "refusal": null
}


In [40]:
response=completion.choices[0].message

In [41]:
response

<OpenAIObject at 0x184ccd951c0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\":\"Bangalore\"}"
  },
  "refusal": null
}

In [42]:
function_name=response['function_call']['name']
print(function_name)


get_current_weather


In [43]:
import json
location=eval(response['function_call']['arguments'])['location']
print(location)

Bangalore


In [44]:
# Step 4: send the info on the function call and function response to GPT
messages.append(response)  # extend conversation with assistant's reply
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": location,
    }
)

In [45]:
messages

[{'role': 'user', 'content': 'Hi There'},
 {'role': 'user', 'content': 'What is the temperature of Delhi'},
 <OpenAIObject at 0x184ccd85580> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\"location\":\"Delhi\"}"
   },
   "refusal": null
 },
 {'role': 'function', 'name': 'get_current_weather', 'content': 'Delhi'},
 {'role': 'user', 'content': 'What is the temperature of Mumbai'},
 <OpenAIObject at 0x184ccd874c0> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\"location\":\"Mumbai\"}"
   },
   "refusal": null
 },
 {'role': 'function', 'name': 'get_current_weather', 'content': 'Mumbai'},
 {'role': 'user', 'content': 'What is the temperature of Bangalore'},
 <OpenAIObject at 0x184ccd951c0> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\"locati

In [46]:
# extend conversation with function response
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)  # get a new response from GPT where it can see the function response


In [47]:
print(second_response.choices[0].message)

{
  "role": "assistant",
  "content": "Currently, the weather in Bangalore is clear with a temperature of 28\u00b0C.",
  "refusal": null
}
